<a href="https://colab.research.google.com/github/kylelee417/Asphalt_Crack_Segmentation/blob/master/Asphalt_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as keras


Using TensorFlow backend.


In [0]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
train_ori = '/content/drive/My Drive/Asphalt_Masked/train_frames/'
train_mask = '/content/drive/My Drive/Asphalt_Masked/train_masks/'

test_ori = '/content/drive/My Drive/Asphalt_Masked/test_frames/'
test_mask ='/content/drive/My Drive/Asphalt_Masked/test_masks/'


In [0]:
img_width = 96
img_height = 96
img_channel = 1
input_shape = (img_height,img_width,img_channel)
epochs = 50
batch_size = 32

In [13]:
train_ori_generator = train_datagen.flow_from_directory(train_ori,
                                                    target_size=(img_width,img_height),
                                                    batch_size=batch_size,
                                                    class_mode=None,
                                                    color_mode='grayscale',
                                                    shuffle=True,
                                                    seed=0)
train_mask_generator = train_datagen.flow_from_directory(train_mask,
                                                    target_size=(img_width,img_height),
                                                    class_mode=None,
                                                    color_mode='grayscale', 
                                                    batch_size=batch_size, 
                                                    shuffle=True,
                                                    seed=0)
test_ori_generator = test_datagen.flow_from_directory(test_ori,
                                                    target_size=(img_width,img_height),
                                                    batch_size=batch_size,
                                                    class_mode=None,
                                                    color_mode='grayscale',
                                                    shuffle=True,
                                                    seed=0)
test_mask_generator = test_datagen.flow_from_directory(test_mask,
                                                    target_size=(img_width,img_height),
                                                    class_mode=None,
                                                    color_mode='grayscale', 
                                                    batch_size=batch_size, 
                                                    shuffle=True,
                                                    seed=0)



Found 1920 images belonging to 1 classes.
Found 1920 images belonging to 1 classes.
Found 480 images belonging to 1 classes.
Found 480 images belonging to 1 classes.


In [0]:
train_generator = zip(train_ori_generator,train_mask_generator)
test_generator = zip(test_ori_generator,test_mask_generator)

In [0]:
def unet_build():
    
    inputs = Input(shape=input_shape)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs,conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])


    return model

In [21]:
m = unet_build()
m.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 96, 96, 1)    0                                            
__________________________________________________________________________________________________
conv2d_97 (Conv2D)              (None, 96, 96, 64)   640         input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_98 (Conv2D)              (None, 96, 96, 64)   36928       conv2d_97[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_17 (MaxPooling2D) (None, 48, 48, 64)   0           conv2d_98[0][0]                  
____________________________________________________________________________________________

In [30]:
hist = m.fit_generator(train_generator, epochs=epochs, 
                       steps_per_epoch=3840//32, 
                       validation_data = test_generator, 
                       validation_steps = 960//32, 
                       verbose=1)

Epoch 1/50
120/120 [==============================] - 762s 6s/step - loss: 0.2146 - acc: 0.9315 - val_loss: 0.2319 - val_acc: 0.9539
Epoch 2/50
120/120 [==============================] - 45s 377ms/step - loss: 0.1727 - acc: 0.9349 - val_loss: 0.1958 - val_acc: 0.9538
Epoch 3/50
120/120 [==============================] - 44s 367ms/step - loss: 0.1550 - acc: 0.9418 - val_loss: 0.2083 - val_acc: 0.9541
Epoch 4/50
120/120 [==============================] - 45s 373ms/step - loss: 0.1408 - acc: 0.9462 - val_loss: 0.1917 - val_acc: 0.9534
Epoch 5/50
120/120 [==============================] - 45s 371ms/step - loss: 0.1300 - acc: 0.9485 - val_loss: 0.2038 - val_acc: 0.9541
Epoch 6/50
120/120 [==============================] - 45s 372ms/step - loss: 0.1184 - acc: 0.9514 - val_loss: 0.2057 - val_acc: 0.9545
Epoch 7/50
120/120 [==============================] - 45s 371ms/step - loss: 0.1120 - acc: 0.9536 - val_loss: 0.2780 - val_acc: 0.9546
Epoch 8/50
 89/120 [=====================>........] - ETA

KeyboardInterrupt: ignored